# Machine Translation With Attention

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

/home/thojo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Verify GPU

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8204871948905942251
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10909243802
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 5879737549639989648
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 8595629671
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 1841046962071392741
physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1"
]


### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file.  I'm reusing Udacity's dataset here, which has been preprocessed already.  The next step is to use a standard dataset such as [WMT](http://www.statmt.org/).

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')
print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  This dataset has been specifically chosen to have a small vocabulary. For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Preprocess
Convert the text into sequences of integers using:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize (IMPLEMENTATION)
Each character or word can be turned into character and word ids (integers), respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity.

Each sentence can be turned into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. 

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """

    x_tk = Tokenizer(char_level=False)
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk

    return None, None
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding 
When batching the sequence of word ids together, each sequence needs to be the same length.  Hence, padding.

All source language sentences should be of the same length and all target language sentences too.  No need for source and target sentences to be of the same length, though. 

In [8]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
All the preprocessing steps put together.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [11]:
# def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
#     """
#     Build and train a basic RNN on x and y
#     :param input_shape: Tuple of input shape
#     :param output_sequence_length: Length of output sequence
#     :param english_vocab_size: Number of unique English words in the dataset
#     :param french_vocab_size: Number of unique French words in the dataset
#     :return: Keras model built, but not trained
#     """
#     # TODO: Build the layers
#     learning_rate = 1e-3

#     input_seq = Input(input_shape[1:])
#     rnn = GRU(128, return_sequences=True)(input_seq)
#     logits = TimeDistributed(Dense(french_vocab_size))(rnn)

#     model = Model(input_seq, Activation('softmax')(logits))
#     model.compile(loss=sparse_categorical_crossentropy,
#                   optimizer=Adam(learning_rate),
#                   metrics=['accuracy'])
#     return model
# tests.test_simple_model(simple_model)

# # Reshaping the input to work with a basic RNN
# tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# # Train the neural network
# simple_rnn_model = simple_model(
#     tmp_x.shape,
#     max_french_sequence_length,
#     english_vocab_size,
#     french_vocab_size)

In [12]:
# simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# # Print prediction(s)
# print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

In [13]:
# simple_rnn_model.save('saved_models/simple_rnn_model.h5')
# del simple_rnn_model

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [14]:
# from keras.layers import Embedding, LSTM

# def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
#     """
#     Build and train a RNN model using word embedding on x and y
#     :param input_shape: Tuple of input shape
#     :param output_sequence_length: Length of output sequence
#     :param english_vocab_size: Number of unique English words in the dataset
#     :param french_vocab_size: Number of unique French words in the dataset
#     :return: Keras model built, but not trained
#     """
#     # Implement model with embedding
#     embed_size = 200 
#     learning_rate = 1e-3
#     input_seq = Input(input_shape[1:])
#     embed = Embedding(english_vocab_size, embed_size, input_length=input_shape[-1])(input_seq)
#     rnn = LSTM(128, return_sequences=True)(embed) # accuracy = 87%
#     logits = TimeDistributed(Dense(french_vocab_size))(rnn)

#     model = Model(input_seq, Activation('softmax')(logits))
#     model.compile(loss=sparse_categorical_crossentropy,
#                   optimizer=Adam(learning_rate),
#                   metrics=['accuracy'])
#     return model

# tests.test_embed_model(embed_model)



# # TODO: Reshape the input. I'm just padding
# tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# # tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# # Train the neural network
# rnn_with_embedding_model = embed_model(
#     tmp_x.shape,
#     max_french_sequence_length,
#     english_vocab_size,
#     french_vocab_size)

In [15]:
# rnn_with_embedding_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# # Print prediction(s)
# print(logits_to_text(rnn_with_embedding_model.predict(tmp_x[:1])[0], french_tokenizer))

In [16]:
# rnn_with_embedding_model.save('saved_models/rnn_with_embedding_model.h5')
# del rnn_with_embedding_model

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [17]:
# from keras.layers import Bidirectional 
# from  keras.models  import Sequential
# from keras.layers import LSTM

# def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
#     """
#     Build and train a bidirectional RNN model on x and y
#     :param input_shape: Tuple of input shape
#     :param output_sequence_length: Length of output sequence
#     :param english_vocab_size: Number of unique English words in the dataset
#     :param french_vocab_size: Number of unique French words in the dataset
#     :return: Keras model built, but not trained
#     """

#     hidden_units = 128
    
#     # GRU implementation (about 76% accuracy)
#     input_seq = Input(input_shape[1:])
#     bd = Bidirectional( GRU( hidden_units, return_sequences=True) )( input_seq )
#     bd = Bidirectional( GRU( hidden_units, return_sequences=True) )( bd ) #this second layer is just for experimenting
#     logits = TimeDistributed( Dense(french_vocab_size) )(bd)
#     model = Model(input_seq, Activation('softmax')(logits))

    
#     # LSTM implementation using Sequential (about 60% accuracy)
# #     model = Sequential()
# #     model.add(Bidirectional(LSTM(hidden_units, return_sequences = True, dropout = 0.2, recurrent_dropout = 0.2),
# #                             input_shape = input_shape[1:]))
# #     model.add(Bidirectional(LSTM(hidden_units, return_sequences = True, dropout = 0.2, recurrent_dropout = 0.2) ))
# #     model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

#     ################
#     learning_rate = 1e-3
#     model.compile(loss=sparse_categorical_crossentropy,
#                   optimizer=Adam(learning_rate),
#                   metrics=['accuracy'])
#     return model
# tests.test_bd_model(bd_model)


# # TODO: Train and Print prediction(s)
# tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# # Train the neural network
# bidirectional_model = bd_model(
#     tmp_x.shape,
#     max_french_sequence_length,
#     english_vocab_size,
#     french_vocab_size)

In [18]:
# bidirectional_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# # Print prediction(s)
# print(logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], french_tokenizer))

In [19]:
# bidirectional_model.save('saved_models/bidirectional_model.h5')
# del bidirectional_model

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [20]:
# from keras.models import Model
# from keras.layers import Input, LSTM, Dense

# def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
#     """
#     Build and train an encoder-decoder model on x and y
#     :param input_shape: Tuple of input shape
#     :param output_sequence_length: Length of output sequence
#     :param english_vocab_size: Number of unique English words in the dataset
#     :param french_vocab_size: Number of unique French words in the dataset
#     :return: Keras model built, but not trained
#     """
#     num_hidden = 128
# #     ###################################### USING LSTMs ####### accuracy about 70%
# #     model = Sequential()
# #     # Encoder
# #     model.add(LSTM(num_hidden, go_backwards=True, input_shape=input_shape[1:])) 
# #     model.add(RepeatVector(output_sequence_length))
# #     # Decoder
# #     model.add(LSTM(num_hidden, return_sequences=True))
# #     model.add(TimeDistributed(Dense(french_vocab_size)))
# #     model.add(Activation('softmax'))    

#     ##################################### USING GRUs ####### accuracy about 75%
#     model = Sequential()
#     # Encoder
#     model.add(GRU(num_hidden, go_backwards=True, input_shape=input_shape[1:]))
#     model.add(RepeatVector(output_sequence_length))
#     # Decoder
#     model.add(GRU(num_hidden, return_sequences=True))
#     model.add(TimeDistributed(Dense(french_vocab_size)))
#     model.add(Activation('softmax'))    

    
#     ##########################
  
#     # Compile model
#     learning_rate = 1e-3
#     model.compile(loss = 'sparse_categorical_crossentropy',
#                  optimizer = Adam(learning_rate),
#                  metrics = ['accuracy'])
#     return model
# tests.test_encdec_model(encdec_model)
# # OPTIONAL: Train and Print prediction(s)
# tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# # Train the neural network
# ed_model = encdec_model(
#     tmp_x.shape,
#     max_french_sequence_length,
#     english_vocab_size,
#     french_vocab_size)

# print(ed_model.summary())

In [21]:
# ed_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# # Print prediction(s)
# print(logits_to_text( ed_model.predict(tmp_x[:1])[0], french_tokenizer))

In [22]:
# ed_model.save('saved_models/ed_model.h5')
# del ed_model

### Define model architecture

Create a translation model (t_model) that incorporates embedding and a bidirectional rnn into one model.

In [23]:
from keras.layers import Dropout
from keras.models  import Sequential
def t_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    embed_size = 200    
    learning_rate = 1e-3
    num_hidden = 256
    
    model = Sequential()
    # Encoder part
    model.add( Embedding(english_vocab_size, embed_size, input_length=input_shape[-1] ) )    
#     model.add( Bidirectional( LSTM(num_hidden, go_backwards=True, input_shape=input_shape[1:],  dropout = 0.5, recurrent_dropout = 0.5)) )  
    model.add( Bidirectional( GRU(num_hidden, go_backwards=True, dropout = 0.4, recurrent_dropout = 0.4 ) ) )
    model.add( RepeatVector(output_sequence_length) )

    # Decoder part
#     model.add( Bidirectional( LSTM(num_hidden, return_sequences=True, dropout = 0.5, recurrent_dropout = 0.5)) ) 
    model.add( Bidirectional( GRU(num_hidden, return_sequences=True, dropout = 0.4, recurrent_dropout = 0.4 ) ) )
    model.add( TimeDistributed( Dense(512) ) )
    model.add( Dropout(0.5))
    model.add( TimeDistributed( Dense(french_vocab_size) ) )
    model.add( Activation('softmax') )

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_model_final(t_model)
print('Final Model Loaded')

Final Model Loaded


/home/thojo/.local/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [24]:
# Train the final model
tmp_x = pad( preproc_english_sentences, max_french_sequence_length )
tmp_x = tmp_x.reshape( (-1, preproc_french_sentences.shape[-2], 1) )

my_model = t_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print( my_model.summary() )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 200)            39800     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               701952    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 21, 512)           1181184   
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 512)           262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 21, 512)           0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 344)           176472    
__________

## Prediction (IMPLEMENTATION)

In [27]:
def final_predictions( x, y, x_tk, y_tk ):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # note: x and y are already pre-processed
    # Train neural network using model_final
    model = t_model(x.shape, 
                    y.shape[1], 
                    len(x_tk.word_index), 
                    len(y_tk.word_index) )
    
    model.fit( x, y, batch_size=2048, epochs=10, validation_split=0.2 )
    
    y_id_to_word = { value: key for key, value in y_tk.word_index.items() }
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [ x_tk.word_index[word] for word in sentence.split() ]
    sentence = pad_sequences( [sentence], maxlen=x.shape[-1], padding='post' )
    sentences = np.array( [sentence[0], x[0]] )
    predictions = model.predict( sentences, len(sentences ) )

    print('Sample 1:')
    print(' '.join( [y_id_to_word[np.argmax(x)] for x in predictions[0]] ) )
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join( [y_id_to_word[np.argmax(x)] for x in predictions[1]] ) )
    print(' '.join( [y_id_to_word[np.max(x)] for x in y[0]] ) )
    return model

In [41]:
full_model=final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 15s 136us/step - loss: 3.1629 - acc: 0.4300 - val_loss: nan - val_acc: 0.4826
Epoch 2/10
110288/110288 [==============================] - 11s 102us/step - loss: 2.1792 - acc: 0.4799 - val_loss: nan - val_acc: 0.5059
Epoch 3/10
110288/110288 [==============================] - 11s 103us/step - loss: 1.9023 - acc: 0.5226 - val_loss: nan - val_acc: 0.5949
Epoch 4/10
110288/110288 [==============================] - 11s 102us/step - loss: 1.6121 - acc: 0.5796 - val_loss: nan - val_acc: 0.6376
Epoch 5/10
110288/110288 [==============================] - 11s 102us/step - loss: 1.4246 - acc: 0.6162 - val_loss: nan - val_acc: 0.6703
Epoch 6/10
110288/110288 [==============================] - 11s 103us/step - loss: 1.2822 - acc: 0.6435 - val_loss: nan - val_acc: 0.7045
Epoch 7/10
110288/110288 [==============================] - 11s 103us/step - loss: 1.1613 - acc: 0.6720 - val_loss: nan -

In [42]:
full_model.save( 'saved_models/full_model.h5' )

# Notes:

-  I have used `go_backwards=True` for the encoding RNNs in the Encoder-Decoder model as well as the final_model per suggestions in places such as __[this link](https://cntk.ai/pythondocs/CNTK_204_Sequence_To_Sequence.html)__ which advice to use `False` for the attention models and `True` otherwise. However, I found that the models here also work with the default `go_backwards=False` but just have lower accuracy by about 5%. 
-  References for encoder-decoder __[1](https://blog.keras.io/building-autoencoders-in-keras.html)__, __[2](https://github.com/keras-team/keras/issues/5203)__, __[3](https://www.jianshu.com/p/c294e4cb4070)__

## Proper test train split

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( english_sentences, french_sentences, test_size=0.1, random_state=42)

tt_preproc_english_sentences, tt_preproc_french_sentences, tt_english_tokenizer, tt_french_tokenizer =\
    preprocess( X_train, y_train )
    
tt_max_english_sequence_length = tt_preproc_english_sentences.shape[1]
tt_max_french_sequence_length = tt_preproc_french_sentences.shape[1]
tt_english_vocab_size = len(tt_english_tokenizer.word_index)
tt_french_vocab_size = len(tt_french_tokenizer.word_index)

tt_x_pad = pad( tt_preproc_english_sentences, tt_max_french_sequence_length )
tt_x = tt_x_pad.reshape( ( -1, tt_preproc_french_sentences.shape[-2], 1 ) )

test_preproc_english_sentences, test_preproc_french_sentences, _, _ = preprocess( X_test, y_test )
test_x_pad = pad( test_preproc_english_sentences, tt_preproc_french_sentences.shape[1] )
test_x = test_x_pad.reshape( (-1, tt_preproc_french_sentences.shape[1], 1 ) )
test_y = pad( test_preproc_french_sentences, tt_preproc_french_sentences.shape[1] ) 

print( 'Data Preprocessed')
print( "Max English sentence length:", tt_max_english_sequence_length )
print( "Max French sentence length:", tt_max_french_sequence_length )
print( "English vocabulary size:", tt_english_vocab_size )
print( "French vocabulary size:", tt_french_vocab_size )

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [ ]:
# # Train the simple neural network on the training sample alone
# tt_simple_rnn_model = simple_model(    
#     tt_x.shape,
#     tt_max_french_sequence_length,
#     tt_english_vocab_size,
#     tt_french_vocab_size)
# tt_simple_rnn_model.fit( tt_x, tt_preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.1 ) 
# print( logits_to_text( tt_simple_rnn_model.predict(tt_x[:1])[0], tt_french_tokenizer ) )

In [ ]:
# #code sample to load and test a model
# from keras.models import load_model
# simple_rnn_model = load_model('saved_models/simple_rnn_model.h5')
# print(logits_to_text(simple_rnn_model.predict(tmp_x[:10])[0], french_tokenizer))

In [ ]:
# score, acc = tt_simple_rnn_model.evaluate( test_x, test_y, batch_size=128 )
# print( 'simple model true score = {:4.2f}'.format(score) )
# print( 'simple model true acc = {:4.2f}'.format(acc) )

In [ ]:
# tt_simple_rnn_model.save('saved_models/tt_simple_rrn_model.h5')
# del tt_simple_rnn_model

The accuracy for the simple RNN model dropped to 0.57 from 0.76 when proper test-train split was done.

# (optional continued) Next we look at proper accuracy evaluated for a bidirectional RNN model

In [ ]:
# tt_ed_model = encdec_model(
#     tt_x.shape,
#     tt_max_french_sequence_length,
#     tt_english_vocab_size,
#     tt_french_vocab_size)
# tt_ed_model.fit( tt_x, tt_preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.1 ) 
# print( logits_to_text( tt_ed_model.predict( tt_x[:1] )[0], tt_french_tokenizer ) )

In [ ]:
# score, acc = tt_ed_model.evaluate(test_x, test_y, batch_size=128)
# print( 'Encoder-decoder model true score = {:4.2f}'.format(score) )
# print( 'Encoder-decoder model acc = {:4.2f}'.format(acc) )

In [ ]:
# tt_ed_model.save( 'saved_models/tt_ed_model.h5' )
# del tt_ed_model

# Accuracy on the test set
Implement test-train split

In [44]:
tt_model = t_model(
    tt_x_pad.shape,
    tt_max_french_sequence_length,
    tt_english_vocab_size,
    tt_french_vocab_size)
tt_model.fit( tt_x_pad, tt_preproc_french_sentences, batch_size=4096, epochs=20, validation_split=0.1 ) 

Train on 111666 samples, validate on 12408 samples
Epoch 1/20
111666/111666 [==============================] - 15s 136us/step - loss: 3.7275 - acc: 0.3961 - val_loss: nan - val_acc: 0.4400
Epoch 2/20
111666/111666 [==============================] - 11s 100us/step - loss: 2.5907 - acc: 0.4623 - val_loss: nan - val_acc: 0.4662
Epoch 3/20
111666/111666 [==============================] - 11s 100us/step - loss: 2.2249 - acc: 0.4749 - val_loss: nan - val_acc: 0.4784
Epoch 4/20
111666/111666 [==============================] - 11s 100us/step - loss: 2.0915 - acc: 0.4887 - val_loss: nan - val_acc: 0.5237
Epoch 5/20
111666/111666 [==============================] - 11s 100us/step - loss: 1.9239 - acc: 0.5212 - val_loss: nan - val_acc: 0.5798
Epoch 6/20
111666/111666 [==============================] - 11s 100us/step - loss: 1.7265 - acc: 0.5577 - val_loss: nan - val_acc: 0.6076
Epoch 7/20
111666/111666 [==============================] - 11s 100us/step - loss: 1.5922 - acc: 0.5847 - val_loss: nan -

In [45]:
print( logits_to_text( tt_model.predict( tt_x_pad[:1] )[0], tt_french_tokenizer) )

la mangue est son fruit préféré mais le raisin est son favori <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [46]:
score, acc = tt_model.evaluate( test_x_pad, test_y, batch_size=128 )
print( 'Final model true score = {:4.2f}'.format(score) )
print( 'Final model acc = {:4.2f}'.format(acc) )

13787/13787 [==============================] - 5s 344us/step
Final model true score = 3.55
Final model acc = 0.53


In [47]:
tt_model.save( 'saved_models/tt_model.h5' )

The accuracy for the final model dropped to 0.53 from 0.96 when proper test-train split was done.